In [55]:
from IPython.display import HTML, display
from sklearn.metrics import accuracy_score

In [56]:
def load_data(year="2017", airport=None):
    data=pandas.read_csv("./data/processed/"+year+".csv")
    # define delay as DepDelay > 30 or ArrDelay>30 -- which implies traveler anxious at airport or miss connection flight
    data["Delay"] = ((data["DepDelay"] > 30) | (data["ArrDelay"] > 30))*1

    # discretize ArrTime, DepTime, AirTime,Distance
    # ArrTime, DepTime are divided by 100, because 22:30 is sored as 2230.
    # AirTime is divided into hours
    # Distance is divided by 250
    data = data.sort_values(['Year', 'Month','DayofMonth','CRSDepTime'], ascending=[True, True, True, True])
    data["ArrTime"] = data["ArrTime"]//100
    data["DepTime"] = data["DepTime"]//100
    data["AirTime"] = data["AirTime"]//60
    data["Distance"] = data["Distance"] // 250
    label_set = pandas.factorize(data["UniqueCarrier"])
    data["UniqueCarrier"]=label_set[0]
    # for keep data from certain airport

    if airport:
        data = data.query("Origin==\'"+airport+"\'")
        # data = data[data["Origin"]=="IAH"]
    data = data.drop("Dest",1)
    data = data.drop("Origin",1)
    data = data.drop("Quarter",1)
    display(data.head())
    return data

# utility for model scoring
# the model has two states, 0 or 1. The state 0 and 1 may mean different things for each run, so it may need to be flipped.
def score_model(model, data, truth, flip=False):
    if flip:        
        return accuracy_score(model.predict(data),truth)
    else:
        return 1 - accuracy_score(model.predict(data),truth)
# score the model given validation serires
def score_model_given_series(model, validate_x, validate_y):
    # is 0 delay or is 1 delay
    zero_score = score_model(model, validate_x[0], validate_y[0])
    one_score = score_model(model, validate_x[0], validate_y[0], True)
    flip = None
#     if zero_score > one_score:
#         flip = False
#     else:
#         flip = True
    sum_score = 0
    for i in range(len(validate_x)):
        score = score_model(model, validate_x[i], validate_y[i], flip)
        sum_score += score
    print(sum_score/len(validate_x))

In [ ]:
def parser(x):
    return datetime.strptime('200'+x, '%Y-%M')
 
# series = read_csv('~/Downloads/sales-of-shampoo-over-a-three-ye.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
series = load_data("2017", "IAH")
X = series.Delay.astype(float)
size = int(len(X) * 0.66)
train, test = X[0:size].values, X[size:len(X)].values
history = [x for x in train]
predictions = list()

for t in range(len(test)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()

,Year,Month,DayofMonth,DayOfWeek,UniqueCarrier,FlightNum,OriginAirportID,DestAirportID,ArrTime,AirTime,Distance,DepDelay,ArrDelay,DepTime,CRSDepTime,Cancelled,Delay
226729,2017,1,1,7,1,1044,12266,13303,8.0,1.0,3.0,-10.0,24.0,4.0,505,0.0,0
414988,2017,1,1,7,3,1499,12266,13930,7.0,1.0,3.0,-3.0,-32.0,5.0,515,0.0,0
414382,2017,1,1,7,3,513,12266,11292,7.0,1.0,3.0,10.0,-15.0,5.0,540,0.0,0
820,2017,1,1,7,1,540,12266,14107,7.0,2.0,4.0,-3.0,-7.0,5.0,600,0.0,0
20645,2017,1,1,7,4,918,12266,10397,8.0,1.0,2.0,-1.0,-5.0,5.0,600,0.0,0


predicted=-0.000000, expected=0.000000
predicted=-0.000000, expected=0.000000
predicted=-0.000000, expected=1.000000
predicted=0.174477, expected=0.000000
predicted=0.164660, expected=0.000000
predicted=0.171623, expected=0.000000
predicted=0.174297, expected=0.000000
predicted=0.158817, expected=0.000000
predicted=0.156254, expected=0.000000
predicted=-0.000000, expected=0.000000
predicted=-0.000000, expected=0.000000
predicted=-0.000000, expected=1.000000
predicted=0.174475, expected=0.000000
predicted=0.164661, expected=0.000000
predicted=0.171622, expected=0.000000
predicted=0.174296, expected=0.000000
predicted=0.158818, expected=0.000000
predicted=0.156255, expected=0.000000
predicted=-0.000000, expected=0.000000
predicted=-0.000000, expected=0.000000
predicted=-0.000000, expected=0.000000
predicted=-0.000000, expected=0.000000
predicted=-0.000000, expected=0.000000
predicted=-0.000000, expected=0.000000
predicted=-0.000000, expected=0.000000
predicted=-0.000000, expected=1.00000